In [33]:
import cv2
import numpy as np
import os

# Ruta de la carpeta con las imágenes
carpeta_entrada = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/arco_superior/"
carpeta_salida = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/"  # Carpeta donde guardar las imágenes con los contornos

# Crear la carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Listar las imágenes en la carpeta
imagenes = sorted([f for f in os.listdir(carpeta_entrada) if f.endswith(".jpeg")])

# Definir los límites de color para detectar la caja roja (ajusta estos valores si es necesario)
bajo_rojo = np.array([0, 0, 100])  # Rojo en el canal BGR
alto_rojo = np.array([100, 100, 255])  # Rojo en el canal BGR

for img_nombre in imagenes:
    # Leer la imagen
    img_path = os.path.join(carpeta_entrada, img_nombre)
    img = cv2.imread(img_path)

    # Convertir la imagen a espacio de color HSV para detectar el color rojo
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Crear una máscara para la detección del rojo
    mascara_roja = cv2.inRange(hsv_img, bajo_rojo, alto_rojo)
    
    # Encontrar los contornos en la máscara roja
    contornos_rojos, _ = cv2.findContours(mascara_roja, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contorno in contornos_rojos:
        # Obtenemos un rectángulo delimitador de la caja roja
        x, y, w, h = cv2.boundingRect(contorno)
        
        # Recortar la imagen solo a la región de interés (la caja roja)
        roi = img[y:y+h, x:x+w]
        
        # Convertir la región recortada a escala de grises
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        # Aplicar un umbral alto para segmentar los dientes (ajustar el valor si es necesario)
        _, umbral = cv2.threshold(roi_gray, 200, 255, cv2.THRESH_BINARY)
        
        # Encontrar contornos en la región segmentada
        contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Crear imagen negra del mismo tamaño que la ROI para dibujar los contornos
        contorno_img = np.zeros_like(roi_gray)
        cv2.drawContours(contorno_img, contornos, -1, (255), thickness=1)

        # Superponer los contornos en la imagen original
        img[y:y+h, x:x+w, 0] = cv2.bitwise_or(img[y:y+h, x:x+w, 0], contorno_img)  # Canal B
        img[y:y+h, x:x+w, 1] = cv2.bitwise_or(img[y:y+h, x:x+w, 1], contorno_img)  # Canal G
        img[y:y+h, x:x+w, 2] = cv2.bitwise_or(img[y:y+h, x:x+w, 2], contorno_img)  # Canal R

    # Guardar la imagen con los contornos
    output_path = os.path.join(carpeta_salida, img_nombre)
    cv2.imwrite(output_path, img)

    print(f"✅ Imagen guardada con contornos: {output_path}")

print("🚀 Procesamiento completado. Las imágenes con los contornos están en la carpeta 'detections/contornos'.")

✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0115.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0116.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0117.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0118.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0119.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0120.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0121.jpeg
✅ Imagen guardada con contornos: C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/contornos/image0122.jpeg
✅ Imagen

In [36]:
import cv2
import numpy as np
import os

# Ruta de la carpeta con las imágenes
carpeta_entrada = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/arco_superior/"
# Ruta donde se guardará el volumen 3D
carpeta_salida = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/caso19/detections/volumen_3d/"

# Crear la carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Listar las imágenes en la carpeta
imagenes = sorted([f for f in os.listdir(carpeta_entrada) if f.endswith(".jpeg")])

# Definir los límites de color para detectar la caja roja (ajusta estos valores si es necesario)
bajo_rojo = np.array([0, 0, 100])  # Rojo en el canal BGR
alto_rojo = np.array([100, 100, 255])  # Rojo en el canal BGR

# Inicializar la lista para almacenar el volumen 3D
volumen_3d = []

for img_nombre in imagenes:
    # Leer la imagen
    img_path = os.path.join(carpeta_entrada, img_nombre)
    img = cv2.imread(img_path)

    # Convertir la imagen a espacio de color HSV para detectar el color rojo
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Crear una máscara para la detección del rojo
    mascara_roja = cv2.inRange(hsv_img, bajo_rojo, alto_rojo)
    
    # Encontrar los contornos en la máscara roja
    contornos_rojos, _ = cv2.findContours(mascara_roja, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Crear una máscara final para la imagen segmentada
    mascara_segmentada = np.zeros_like(img[:, :, 0])  # Usamos el canal de la imagen para la máscara binaria

    for contorno in contornos_rojos:
        # Obtenemos un rectángulo delimitador de la caja roja
        x, y, w, h = cv2.boundingRect(contorno)
        
        # Recortar la imagen solo a la región de interés (la caja roja)
        roi = img[y:y+h, x:x+w]
        
        # Convertir la región recortada a escala de grises
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
        # Aplicar un umbral alto para segmentar los dientes (ajustar el valor si es necesario)
        _, umbral = cv2.threshold(roi_gray, 200, 255, cv2.THRESH_BINARY)
        
        # Sumar la región segmentada al volumen 3D
        mascara_segmentada[y:y+h, x:x+w] = np.maximum(mascara_segment


_IncompleteInputError: incomplete input (291463818.py, line 54)